In [79]:
import pandas as pd
import numpy as np
import json

In [51]:
# read the text file
f = open("vulnerabilities.txt", "r")

In [52]:
# make a list frome all the vulnerabilities
column_name = []
column_name.append('attacks')

for line in f:
    item = line.replace("\n", "")
    column_name.append(item)
    
print(f"read {len(column_name)} objects.")

read 113 objects.


In [53]:
# add the list to dataframe as columns
dataframe = pd.DataFrame(columns = column_name)
dataframe.to_csv('vulnerabilities.csv',  
                  index = None) 

In [54]:
dataframe

,attacks,broken object level authorization,broken authentication,broken object property level authorization,unrestricted resource consumption,broken function level authorization,unrestricted access to sensitive business flows,server side request forgery,security misconfiguration,improper inventory management,...,unrestricted api endpoints,content spoofing,insecure third party libraries,no header token,authentication lack,multi resource,http query,http path,request path,request input


In [80]:
files = [
    'v0.4/001.json',
]

data = []

for name in files:
    with open(name, 'r') as file:
        data += json.load(file)

In [81]:
print(len(data))

1000


In [82]:
for i in range(len(data)):
    # extract attacks and vulnerabilities
    obj = data[i]
    att = obj['attack']
    vul = obj['vulenrabilities']
    
    # create a full zero array
    arr = np.zeros(len(column_name) - 1)
    
    # for each vulnerability
    for item in vul:
        if item in dataframe.columns:
            # find index
            index = dataframe.columns.tolist().index(item) - 1
            arr[index] = 1
    
    row = []
    row.append(att)
    row = row + arr.tolist()
    
    # add a new row to dataset
    dataframe.loc[-1] = row
    dataframe.index = dataframe.index + 1
    dataframe = dataframe.sort_index()

In [83]:
df_exploded = dataframe.explode('attacks')    

In [84]:
# check dataset
df_exploded.head()

,attacks,broken object level authorization,broken authentication,broken object property level authorization,unrestricted resource consumption,broken function level authorization,unrestricted access to sensitive business flows,server side request forgery,security misconfiguration,improper inventory management,...,unrestricted api endpoints,content spoofing,insecure third party libraries,no header token,authentication lack,multi resource,http query,http path,request path,request input
0,rbac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,payload,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,payload,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dos,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,payload,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
df_exploded['attacks'].value_counts()

attacks
slowloris         284
dos               274
authentication    254
ddos              250
brute-force       242
payload           240
rbac              238
injection         218
Name: count, dtype: int64

In [86]:
# get info of the dataframe
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 0 to 1999
Columns: 113 entries, attacks to request input
dtypes: float64(112), object(1)
memory usage: 1.7+ MB


In [87]:
df_exploded.isnull().values.any()

False

# Test and Train

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
# Splitting data into train and test sets
X = df_exploded.drop('attacks', axis=1)  # Features
y = df_exploded['attacks']  # Target variable

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# SVM Model

In [111]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [166]:
# Initialize and train an SVM Classifier
svm = SVC(kernel='linear', C=0., random_state=42)  # You can adjust the kernel and C value
svm.fit(X_train, y_train)

SVC(C=0.8, kernel='linear', random_state=42)

In [167]:
# Predict on the test set
predictions = svm.predict(X_test)

In [168]:
# Evaluate the model
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

authentication       0.93      0.96      0.95        84
   brute-force       0.85      0.97      0.91        79
          ddos       0.84      0.82      0.83        77
           dos       0.83      0.94      0.88        84
     injection       0.98      0.75      0.85        63
       payload       0.97      0.95      0.96        64
          rbac       0.97      0.94      0.95        65
     slowloris       0.96      0.89      0.93        84

      accuracy                           0.91       600
     macro avg       0.92      0.90      0.91       600
  weighted avg       0.91      0.91      0.91       600



In [169]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

print(f"SVM Model Accuracy: {accuracy:.4f}")

SVM Model Accuracy: 0.9067


# Navie Bayes Model

In [119]:
from sklearn.naive_bayes import MultinomialNB

In [129]:
clf = MultinomialNB(force_alpha=True)

In [130]:
y_pred = clf.fit(X_train, y_train).predict(X_test)
clf.fit(X, y)

MultinomialNB(force_alpha=True)

In [131]:
print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

authentication       0.97      0.90      0.94        84
   brute-force       0.86      0.96      0.91        79
          ddos       0.93      0.74      0.83        77
           dos       0.76      1.00      0.86        84
     injection       1.00      0.83      0.90        63
       payload       0.98      1.00      0.99        64
          rbac       0.91      0.94      0.92        65
     slowloris       0.99      0.92      0.95        84

      accuracy                           0.91       600
     macro avg       0.93      0.91      0.91       600
  weighted avg       0.92      0.91      0.91       600



In [132]:
# Assuming 'svm' is your trained SVM model and 'X_test', 'y_test' are your test data
predictions = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

print(f"CLF Model Accuracy: {accuracy:.4f}")


CLF Model Accuracy: 0.9250
